In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import os
import kagglehub
import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm
import pickle

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
PyTorch version 2.6.0+cpu
Torchvision version 0.21.0+cpu
Numpy version 2.2.4
Pandas version 2.2.3


In [2]:
path = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
pred_path = os.path.join(path, r'TTV\pred')
yes_path = os.path.join(path, r'TTV\yes')
no_path = os.path.join(path, r'TTV\no')

In [8]:
class MRIDataset(Dataset):
    def __init__(self, data_dir, transform = None):
        self.data = ImageFolder(data_dir, transform = transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedhamada0/brain-tumor-detection")

print("Path to dataset files:", path)

100%|██████████| 84.0M/84.0M [00:01<00:00, 51.0MB/s]

Extracting files...


KeyboardInterrupt: 

In [10]:
mri = MRIDataset(
    data_dir ='/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/'
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/'

In [ ]:
#move files from yes folder to valid yes folder 100x, same for nos

#starter = '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/yes/'
#destination = '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/valid/yes/'
#count = 0
#for file in os.listdir(starter):
    #os.replace(os.path.join(starter, file), os.path.join(destination, file))
    #count += 1
    #if count >= 100:
        #break
#destination = '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/valid/no/'
#count = 0
#for file in os.listdir(starter):
    #os.replace(os.path.join(starter, file), os.path.join(destination, file))
    #count += 1
    #if count >= 100:
        #break


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.PILToTensor(),
])

data_dir ='/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/train'

dataset = MRIDataset(data_dir, transform = transform)

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class MRIClassifer(nn.Module):
    def __init__(self, num_classes=53):
        super(MRIClassifer, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280
        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output


In [ ]:
model = MRIClassifer(num_classes=2)
print(str(model)[:500])

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_folder = '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/train/'
valid_folder = '/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/valid/'

train_dataset = MRIDataset(train_folder, transform=transform)
val_dataset = MRIDataset(valid_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Simple training loop
num_epochs = 50
train_losses, val_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = MRIClassifer(num_classes=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        # Move inputs and labels to the device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            # Move inputs and labels to the device
            images, labels = images.to(device), labels.to(device)
         
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")


In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()

In [ ]:
test_image = "/home/jovyan/.cache/kagglehub/datasets/ahmedhamada0/brain-tumor-detection/versions/12/valid/no/no932.jpg"
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

original_image, image_tensor = preprocess_image(test_image, transform)
probabilities = predict(model, image_tensor, device)